In [ ]:
This Code was used to submit on kaggle

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt 

In [3]:
train_data = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
test_data = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
model = keras.models.load_model("/kaggle/input/models/context_concat_model.h5")

In [5]:
from nltk.stem import PorterStemmer
from collections import defaultdict

def bow_preprocess(data_test):
    train_data['anchor_context'] = train_data['anchor'].str.lower() + " " + train_data['context'].str.lower()
    train_data['target_context'] = train_data['target'].str.lower() + " " + train_data['context'].str.lower()
    data_test['anchor_context'] = data_test['anchor'].str.lower() + " " + data_test['context'].str.lower()
    data_test['target_context'] = data_test['target'].str.lower() + " " + data_test['context'].str.lower()

    anchor_context = train_data['anchor_context'].to_numpy()
    target_context = train_data['target_context'].to_numpy()
    anchor_context_ver = data_test['anchor_context'].to_numpy()
    target_context_ver = data_test['target_context'].to_numpy()
    max_anchor_len = max([len(sentence) for sentence in anchor_context])
    max_target_len = max([len(sentence) for sentence in target_context])
    maxlen = max(max_anchor_len, max_target_len)
    # print(maxlen)

    tk = Tokenizer()
    tk.fit_on_texts(np.concatenate((anchor_context, target_context), axis=0))
    anchor_token = tk.texts_to_sequences(anchor_context_ver)
    target_token = tk.texts_to_sequences(target_context_ver)

    def count_num_duplicate_stems(arr, threshold=2):
        _, counts = np.unique(arr, return_counts=True)
        return np.count_nonzero(counts >= threshold)

    X_test_anchor = pad_sequences(anchor_token, maxlen=maxlen, padding='post')
    X_test_target = pad_sequences(target_token, maxlen=maxlen, padding='post')
    ps = PorterStemmer()
    sentences = (data_test['anchor'] + " " + data_test['target']).to_numpy()
    stems = [[ps.stem(w) for w in sentence.split()] for sentence in sentences]
    counts = np.array([count_num_duplicate_stems(stems) for stems in stems]).reshape(-1, 1)
    return X_test_anchor, X_test_target, counts

X_test_anchor, X_test_target, counts = bow_preprocess(test_data)
preds2 = model.predict([X_test_anchor, X_test_target, counts])
preds2 = np.argmax(preds2, axis=1)

print(preds2)


In [6]:
classes = {0:0, 1:0.25, 2:0.50, 3:0.75, 4:1}

predictions = np.array([classes[i] for i in preds2], dtype=object)

print(predictions)

In [8]:
final = pd.DataFrame(dtype=object)
final["id"] = test_data['id']
final['score'] = predictions

final = final.astype(object)

display(final)

print(final.shape)

final.to_csv('/kaggle/working/submission.csv', index=False)